In [1]:
import pandas as pd
import re

In [2]:
tsla = pd.read_csv('TSLA(2).csv')
elon = pd.read_csv('elonmusk.csv')

In [3]:
def separate_date(x):
    timestamp = pd.Timestamp(x)
    date = timestamp.date()
    time = timestamp.time()
    
    return date

In [4]:
def separate_time(x):
    timestamp = pd.Timestamp(x)
    date = timestamp.date()
    time = timestamp.time()

    return time


In [5]:
tsla['Date'] = tsla['Date'].apply(lambda x : pd.Timestamp(x).date())

In [6]:
elon['Date'] = elon.apply(lambda x: separate_date(x['Timestamp']), axis=1)

In [7]:
elon['Time'] = elon.apply(lambda x: separate_time(x['Timestamp']), axis=1)

In [8]:
elon['weekday'] = elon['Date'].apply(lambda x: x.weekday())

In [9]:
elon['is_weekend'] = elon['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)

In [10]:
elon['trading_day'] = 0
elon.loc[elon['Date'].isin(tsla['Date']), ['trading_day']] = 1



In [11]:
def getFivePriorTradingDays(date):
    priors = tsla[tsla['Date'] < date]
    return priors['Date'].tail(5).tolist()
    
    

In [12]:
def getValues(dates, col):
    res = []
    for date in dates:
        res.append(tsla[tsla['Date'] == date][col].values[0])
    
    return res

In [13]:
elon['five_prior_trading_days'] = elon['Date'].apply(lambda x: getFivePriorTradingDays(x))

In [14]:
elon['five_prior_close'] = elon.apply(lambda x: getValues(x['five_prior_trading_days'], 'Close'), axis=1)

In [15]:
elon['five_prior_open'] = elon.apply(lambda x: getValues(x['five_prior_trading_days'], 'Open'), axis=1)


In [17]:
usefulElon = elon[['text','Comments','Likes','Retweets','Date', 'Time','weekday', 'is_weekend', 'five_prior_trading_days','five_prior_open','five_prior_close', 'trading_day']].loc[1:]

In [18]:
usefulElon.head()

,text,Comments,Likes,Retweets,Date,Time,weekday,is_weekend,five_prior_trading_days,five_prior_open,five_prior_close,trading_day
1,Went to Iceland on Sat to ride bumper cars on ...,30,24,188,2011-12-01,09:55:11,3,0,"[2011-11-23, 2011-11-25, 2011-11-28, 2011-11-2...","[6.352, 6.31, 6.4, 6.498, 6.5]","[6.29, 6.332, 6.512, 6.35, 6.548]",1
2,I made the volume on the Model S http://ow.ly/...,29,17,78,2011-12-01,10:29:04,3,0,"[2011-11-23, 2011-11-25, 2011-11-28, 2011-11-2...","[6.352, 6.31, 6.4, 6.498, 6.5]","[6.29, 6.332, 6.512, 6.35, 6.548]",1
3,"Great Voltaire quote, arguably better than Twa...",23,30,44,2011-12-03,08:20:28,5,1,"[2011-11-28, 2011-11-29, 2011-11-30, 2011-12-0...","[6.4, 6.498, 6.5, 6.514, 6.566]","[6.512, 6.35, 6.548, 6.52, 6.66]",0
4,That was a total non sequitur btw\n26\n14\n50,26,14,50,2011-12-03,08:22:07,5,1,"[2011-11-28, 2011-11-29, 2011-11-30, 2011-12-0...","[6.4, 6.498, 6.5, 6.514, 6.566]","[6.512, 6.35, 6.548, 6.52, 6.66]",0
5,Am reading a great biography of Ben Franklin b...,18,18,80,2011-12-04,03:33:52,6,1,"[2011-11-28, 2011-11-29, 2011-11-30, 2011-12-0...","[6.4, 6.498, 6.5, 6.514, 6.566]","[6.512, 6.35, 6.548, 6.52, 6.66]",0


In [19]:
def clean_text(x):
    temp = x.lower()
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    temp = re.sub(r"\n", " ", temp)
    temp = temp.rsplit(' ', 3)[0]
    return temp
    
    
    

In [20]:
usefulElon['clean_text'] = usefulElon['text'].apply(lambda x: clean_text(x))

In [21]:
tsla

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,3.800000,5.000000,3.508000,4.778000,4.778000,93831500
1,2010-06-30,5.158000,6.084000,4.660000,4.766000,4.766000,85935500
2,2010-07-01,5.000000,5.184000,4.054000,4.392000,4.392000,41094000
3,2010-07-02,4.600000,4.620000,3.742000,3.840000,3.840000,25699000
4,2010-07-06,4.000000,4.000000,3.166000,3.222000,3.222000,34334500
...,...,...,...,...,...,...,...
2971,2022-04-18,989.030029,1014.919983,973.409973,1004.289978,1004.289978,17238400
2972,2022-04-19,1005.059998,1034.939941,995.330017,1028.150024,1028.150024,16615900
2973,2022-04-20,1030.000000,1034.000000,975.250000,977.200012,977.200012,23570400
2974,2022-04-21,1074.729980,1092.219971,996.419983,1008.780029,1008.780029,35138800


In [22]:
tsla[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']] = tsla[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].pct_change()
    

In [52]:
tsla

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-06-30,0.357368,0.216800,0.328392,-0.002512,-0.002512,-0.084151
2,2010-07-01,-0.030632,-0.147929,-0.130043,-0.078473,-0.078473,-0.521804
3,2010-07-02,-0.080000,-0.108796,-0.076961,-0.125683,-0.125683,-0.374629
4,2010-07-06,-0.130435,-0.134199,-0.153928,-0.160937,-0.160937,0.336025
...,...,...,...,...,...,...,...
2971,2022-04-18,-0.010267,0.002182,-0.008939,0.019584,0.019584,-0.113433
2972,2022-04-19,0.016208,0.019726,0.022519,0.023758,0.023758,-0.036111
2973,2022-04-20,0.024814,-0.000908,-0.020174,-0.049555,-0.049555,0.418545
2974,2022-04-21,0.043427,0.056306,0.021707,0.032317,0.032317,0.490802


In [81]:
def get_trading_info(row):
    if row['trading_day'] == 1:
        temp = tsla[tsla['Date'] == row['Date']]
    else:
        next_trading_day = tsla[tsla['Date'] > row['Date']].iloc[0]['Date']
        temp = tsla[tsla['Date'] == next_trading_day]
    
    return pd.Series(temp.values[0][1:])

In [82]:
get_trading_info(usefulElon.iloc[0])

0    0.002154
1    0.032189
2   -0.007449
3   -0.004276
4   -0.004276
5    0.354991
dtype: object

In [101]:
labels = usefulElon.copy()
labels[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']] = labels.apply(
    lambda x: get_trading_info(x), axis=1)


In [102]:
labels = labels[['Date','Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

In [103]:
labels

,Date,Open,High,Low,Close,Adj Close,Volume
1,2011-12-01,0.002154,0.032189,-0.007449,-0.004276,-0.004276,0.354991
2,2011-12-01,0.002154,0.032189,-0.007449,-0.004276,-0.004276,0.354991
3,2011-12-03,0.021322,0.038884,0.031790,0.033634,0.033634,0.445067
4,2011-12-03,0.021322,0.038884,0.031790,0.033634,0.033634,0.445067
5,2011-12-04,0.021322,0.038884,0.031790,0.033634,0.033634,0.445067
...,...,...,...,...,...,...,...
12201,2022-03-03,0.007614,-0.000045,-0.013823,-0.046142,-0.046142,-0.174426
12202,2022-03-03,0.007614,-0.000045,-0.013823,-0.046142,-0.046142,-0.174426
12203,2022-03-02,0.002817,-0.003821,-0.011139,0.017955,0.017955,-0.001653
12204,2022-03-04,-0.033763,-0.034734,-0.008936,-0.001191,-0.001191,0.087239


In [104]:
labels.to_csv('lables.csv')
usefulElon.to_csv('features.csv')